In [1]:
import pandas as pd
import pathlib
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Float, String, Date
from sqlalchemy.orm import Session

# Create path object using pathlib
This will ensure this file will run for everyone regardless of os
This group contains folks running on mac, windows, and linux.    

In [2]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Project_3_Group_7':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")

Path is project root


In [3]:
resources_path = pathlib.PurePath(cwd, 'Resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Project_3_Group_7/Resources')

# Load pandas data into memory

## Load aggregate twitch and steam data

In [4]:
steam_twitch_df = pd.read_csv(f"{resources_path}/merge_output.csv")
steam_twitch_df.head(3)

,title,avg_hours_watched,avg_hours_streamed,avg_streamers,avg_viewers,avg_channels,app_id,release_date,reviews_total,review_avg_percent,launch_price_cents,dataset_est_rev_cents
0,.hack//G.U. Last Recode,145350,35258,3174,202,49,525480,2017-11-03,2689,87.0,4999,13442311
1,20 Minutes Till Dawn,911356,12253,3616,1267,17,1966900,2023-06-08,20942,92.0,499,10450058
2,60 Parsecs!,529688,1867,606,736,2,646270,2018-09-18,2315,89.0,999,2312685


In [5]:
tag_df = pd.read_csv(f"{resources_path}/app_id_common_tags.csv")
tag_df.head(3)

,app_id,tag
0,730,FPS
1,730,Shooter
2,730,Multiplayer


In [6]:
prep_steam_twitch = steam_twitch_df.set_index('app_id')
large_df = tag_df.join(prep_steam_twitch, on='app_id', how='inner')
large_df

,app_id,tag,title,avg_hours_watched,avg_hours_streamed,avg_streamers,avg_viewers,avg_channels,release_date,reviews_total,review_avg_percent,launch_price_cents,dataset_est_rev_cents
0,730,FPS,Counter-Strike: Global Offensive,48489362,906847,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
1,730,Shooter,Counter-Strike: Global Offensive,48489362,906847,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
2,730,Multiplayer,Counter-Strike: Global Offensive,48489362,906847,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
3,730,Competitive,Counter-Strike: Global Offensive,48489362,906847,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
4,730,Action,Counter-Strike: Global Offensive,48489362,906847,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106366,968870,Horror,Close to the Sun,343108,3465,867,461,4,2020-05-05,506,75.0,1999,1011494
106367,968870,Violent,Close to the Sun,343108,3465,867,461,4,2020-05-05,506,75.0,1999,1011494
106368,968870,Female Protagonist,Close to the Sun,343108,3465,867,461,4,2020-05-05,506,75.0,1999,1011494
106369,968870,First Person,Close to the Sun,343108,3465,867,461,4,2020-05-05,506,75.0,1999,1011494


## Load monthly twitch data

In [7]:
twitch_monthly_df = pd.read_csv(f"{resources_path}/app_id_twitch_monthly.csv")
twitch_monthly_df.head(3)

,app_id,title,rank,month,year,hours_watched,hours_streamed,peak_viewers,peak_channels,streamers,average_viewers,average_channels
0,525480,.hack//G.U. Last Recode,199,11,2017,145350,35258,1222,258,3174,202,49
1,1966900,20 Minutes Till Dawn,159,6,2022,911356,12253,29743,80,3616,1267,17
2,646270,60 Parsecs!,109,9,2018,529688,1867,31960,27,606,736,2


# Prep sqlalchemy for loading data into db

In [8]:
Base = declarative_base()
# tag_table = "tags"
steam_twitch_tags_table = "steam_twitch_tags"
twitch_time_table = 'twitch_monthly'

/tmp/ipykernel_41394/1664184401.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [9]:
class twitch_monthly(Base):
    __tablename__ = twitch_time_table
    app_id = Column(Integer, primary_key=True)
    title = Column(String(128))
    rank = Column(Integer)
    month = Column(Integer, primary_key=True)
    year = Column(Integer, primary_key=True)
    hours_watched = Column(Integer)
    hours_streamed = Column(Integer)
    peak_viewers = Column(Integer)
    peak_channels = Column(Integer)
    streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)


In [10]:
class combined_twitch_steam_tags(Base):
    __tablename__ = steam_twitch_tags_table
    # https://stackoverflow.com/questions/19129289/how-to-define-composite-primary-key-in-sqlalchemy
    # indicates that this syntax should make a composite primary key
    app_id = Column(Integer, primary_key=True)
    tag = Column(String(128), primary_key=True)
    title = Column(String(128))
    # These two will need to be changed to Float 
    hours_watched = Column(Float)
    hours_streamed = Column(Float)
    average_streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)
    release_date = Column(Date)
    reviews_total = Column(Integer)
    review_avg_percent = Column(Integer)
    launch_price_cents = Column(Integer)
    dataset_est_rev_cents = Column(Integer)



In [11]:
engine = sqlalchemy.create_engine(f'sqlite:///{resources_path}/data.sqlite')
Base.metadata.create_all(engine)

# Add dataframes to DB

In [12]:
conn = engine.connect()

In [13]:
large_df.to_sql(steam_twitch_tags_table, conn, if_exists='replace')

15403

In [14]:
twitch_monthly_df.to_sql(twitch_time_table, conn, if_exists='replace')

5497